In [1]:
import google_sheets_api as gs

In [2]:
df = gs.read_table_to_df()

In [4]:
print(df)

  OrderId TelegramId    Name       Type         Day                 From  \
0          468997622  Андрей  hairstyle  2024-05-31  2024-05-31 10:00:00   
1          468997622  Андрей      brows  2024-06-01  2024-06-01 14:30:00   
2          468997622  Андрей      brows  2024-06-02  2024-06-02 10:30:00   
3          468997622  Андрей  hairstyle  2024-05-30  2024-05-30 10:00:00   
4          468997622  Андрей      brows  2024-07-31  2024-07-31 10:00:00   
5          468997622  Андрей  hairstyle  2024-05-17  2024-05-17 14:30:00   

                    To  Payed  
0  2024-05-31 11:00:00  False  
1  2024-06-01 17:30:00  False  
2  2024-06-02 11:30:00  False  
3  2024-05-30 13:00:00  False  
4  2024-07-31 13:00:00  False  
5  2024-05-17 16:30:00  False  
